# Patch

## Patch name

Suppose you have service that answers some information about user. Just like the function in the next cell:

In [ ]:
import requests

def request_user(user_id):
    response = requests.get(f"https://im_not_exist/{user_id}")
    if response.ok:
        return response.text
    else:
        return "Fail!"

Now we have to test this code. But you don't need to rely on any particular service in your tests, and you know that the requesting function under consideration uses `requests.get` to access information about the user from the service. So the best option for you here is to mock the `requests.get` function. So for test you change it behaviour in benificial for you way.

To do this, use the following syntax `unittest.mock.patch("<module under tests>.<name of the function to mock>")` as an object for the context manager. You must specify a module to change the behaviour of the function for that module only.

So such an option is implemented in the following cell. We've replaced the `request.get` function in the `request_user` module for test purposes. And check with which arguments it was called with and specify the properties of its output for particular case.

In [ ]:
import unittest
from unittest.mock import patch

class TestUserReques(unittest.TestCase):
    
    def test_success_case(self):
        '''
        Sucessfull case. Suppose we got nice
        repsonse from requests.get in request_user function.
        '''
        with patch("__main__.requests.get") as mocked_get:
            # set that output of the request_user.requests.get
            # will have properties `ok == True` `text=="Success`
            mocked_get.return_value.ok = True
            mocked_get.return_value.text = "Success"
            
            response = request_user("Fedor")
            # request_user should have called `get` with specific url
            mocked_get.assert_called_with("https://im_not_exist/Fedor")
            # and "text" property have to be just like we specified
            self.assertEqual(response, "Success")

    def test_fail_case(self):
        '''
        Fail case. Suppose we got bad response
        from requests.get in the request_user function.
        '''
        with patch("__main__.requests.get") as mocked_get:
            # set that output of the request_user.requests.get
            # will have properties `ok == False` `text=="Success`
            # actually we don't really care wich exactly value
            # takes `return_value.text` - `return_value.ok==False`
            # will make answer "Fail!" anyway
            mocked_get.return_value.ok = False
            mocked_get.return_value.text = "Success"
            
            response = request_user("Ekaterina")
            # request_user should have called `get` with specific url
            mocked_get.assert_called_with("https://im_not_exist/Ekaterina")
            # but answer anyway have to be "Fail!" 
            self.assertEqual(response, "Fail!")

ans = unittest.main(argv=[''], verbosity=2, exit=False)
del TestUserReques

test_fail_case (__main__.TestUserReques)
Fail case. Suppose we got bad response ... ok
test_success_case (__main__.TestUserReques)
Sucessfull case. Suppose we got nice ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


Fail!
Success


## Patch class method

Due to the specifics of the tasks I face I often had to make classes that somehow turn around machine learning models. That is, there was a class that contained the model, it should prepare data to feed into the model and process its results.

However, this pattern can occur in other variations. Here we will consider the peculiarities of working with it.

### Known type

In the following cell, such a pattern has just been re-created schematically. And for this case it's important that I know type of the `ClassUnderConsider.obj`.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

class ClassUnderConsider():
    def __init__(self):
        # model creating
        # it isn't necessary to fit it somehow
        # because we will change it's behaviour
        # anyway
        self.obj = KNeighborsRegressor()

    
    def explode_object(self):
        # we will later change behavior of the `predict`
        # method so there aren't any sence to care
        # about arguments it taking
        ans = self.obj.predict()
        # just returning of `predict` outputs
        # we will replace it outputs as well
        return ans

Now test for such a case. We need to make `obj.predict` behave as we need it to during the test run. So `obj` is an instance of the `sklearn.neighbours.KNeighborsRegressor` class, so we can mock it's `predict` method and everything will be fine.

Exactly such a scheme is implemented in the following cell:

In [ ]:
import unittest
from unittest.mock import patch

class ClassTester(unittest.TestCase):
    def test_explode_object(self):
        testing_instance = ClassUnderConsider()
        # mocking method under consideration
        with patch("__main__.KNeighborsRegressor.predict") as mocked_predict:

            # now make it return what we are interested in
            exp_ans = "Something predicted"
            mocked_predict.return_value = exp_ans

            # lets execute method that is udner testing
            # and print what it return - it have to be our
            # exp_ans
            real_ans = testing_instance.explode_object()
            print("Value from predict -", real_ans)

            # and finally chekc of the test
            self.assertEqual(exp_ans, real_ans)

ans = unittest.main(argv=[''], verbosity=2, exit=False)
del ClassTester

test_explode_object (__main__.ClassTester) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Value from predict - Something predicted


"Something predicted" was printed that just how we mocked `KNeighborsRegressor.predict`.

### Unknown type

It's also popular when different types of objects can be wrapped. So suppose object of the model passed as external object. Just like example below:

In [ ]:
class ClassUnderConsider():
    def __init__(self, obj):
        # so we got external object that
        # with unknown type
        self.obj = obj

    
    def explode_object(self):
        ans = self.obj.predict()
        return ans

**Intuitive but wrong**

It's very tempting to make an attempt to change behaviour of the `ClassUnderConsider.obj.predict` but it won't work. In the following cell I'll show it.

In [ ]:
import unittest
from unittest.mock import patch
from sklearn.neighbors import KNeighborsRegressor

class ClassTester(unittest.TestCase):
    def test_explode_object(self):
        testing_instance = ClassUnderConsider(KNeighborsRegressor())
        # here in `patch` I'm trying to refer to the
        # __main__.ClassUnderConsider.obj.predict
        # but it won't work
        with patch("__main__.ClassUnderConsider.obj.predict") as mocked_predict:
            exp_ans = "Something predicted"
            mocked_predict.return_value = exp_ans

            real_ans = testing_instance.explode_object()
            print("Value from predict -", real_ans)

            self.assertEqual(exp_ans, real_ans)


ans = unittest.main(argv=[''], verbosity=2, exit=False)
del ClassTester

test_explode_object (__main__.ClassTester) ... ERROR

ERROR: test_explode_object (__main__.ClassTester)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_23027/691487794.py", line 11, in test_explode_object
    with patch("__main__.ClassUnderConsider.obj.predict") as mocked_predict:
  File "/usr/lib/python3.10/unittest/mock.py", line 1431, in __enter__
    self.target = self.getter()
  File "/usr/lib/python3.10/unittest/mock.py", line 1618, in <lambda>
    getter = lambda: _importer(target)
  File "/usr/lib/python3.10/unittest/mock.py", line 1261, in _importer
    thing = _dot_lookup(thing, comp, import_path)
  File "/usr/lib/python3.10/unittest/mock.py", line 1250, in _dot_lookup
    __import__(import_path)
ModuleNotFoundError: No module named '__main__.ClassUnderConsider'; '__main__' is not a package

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (error

So we got an error saying that it's not possible to use `"__main__.ClassUnderConsider.obj.predict"` inside the `patch` function.

**Possible solution**

The best option found here is just create fake class that is just behave like expected in test to behave and don't use any features fo the `unittest`.

In [ ]:
import unittest
from unittest.mock import patch
from sklearn.neighbors import KNeighborsRegressor

class ClassTester(unittest.TestCase):
    def test_explode_object(self):
        
        exp_ans = "Something predicted"
        # here is class that have all necessary for
        # `ClassUnderConsider` methods, and I know
        # how exactly it works
        class ModelImitator():
            # it's predict just return expected value
            def predict(self): return exp_ans

        testing_instance = ClassUnderConsider(ModelImitator())

        real_ans = testing_instance.explode_object()
        print("Value from predict -", real_ans)
        self.assertEqual(exp_ans, real_ans)

ans = unittest.main(argv=[''], verbosity=2, exit=False)
del ClassTester

test_explode_object (__main__.ClassTester) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Value from predict - Something predicted


Now it's outputs just like in case with known type of the object.